# Deep Transition Dependency Parser in PyTorch

In this problem set, you will implement a deep transition dependency parser in [PyTorch](https://pytorch.org).

You will:

- Implement an arc-standard transition-based dependency parser in PyTorch
- Implement neural network components for choosing actions and combining stack elements
- Train your network to parse English and Norwegian sentences
- Implement techniques to improve your parser

# 0. Setup

In order to develop this assignment, you will need [python 3](https://www.python.org/downloads/) and the following libraries. Most if not all of these are part of [conda](https://docs.conda.io/en/latest/miniconda.html), so a good starting point would be to install that.

- [jupyter](http://jupyter.readthedocs.org/en/latest/install.html)
- [numpy](https://docs.scipy.org/doc/numpy/user/install.html)
- [matplotlib](http://matplotlib.org/users/installing.html)
- [nosetests](https://nose.readthedocs.org/en/latest/)
- [pytorch](https://pytorch.org/get-started/locally/)

Here is some help on installing packages in python: https://packaging.python.org/installing/. You can use ```pip --user``` to install locally without sudo.

## About this assignment

- This is a Jupyter notebook. You can execute cell blocks by pressing control-enter.
- Most of your coding will be in the python source files in the directory ```mynlplib```.
- The directory ```tests``` contains unit tests that will be used to grade your assignment, using ```nosetests```. You should run them as you work on the assignment to see that you're on the right track. You are free to look at their source code, if that helps -- though most of the relevant code is also here in this notebook. Learn more about running unit tests at https://nose.readthedocs.io/en/latest/usage.html.
- You may want to add more tests, but that is completely optional.
- **To submit this assignment, run the script ```make-submission.sh```, and submit the tarball ```pset3-submission.tgz``` on Canvas.**

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as ag

import nose
import numpy as np

from imp import reload

In [2]:
print('My library versions')

print('numpy: {}'.format(np.__version__))
print('nose: {}'.format(nose.__version__))
print('torch: {}'.format(torch.__version__))

My library versions
numpy: 1.19.1
nose: 1.3.7
torch: 1.6.0


To test whether your libraries are the right version, run:

`nosetests tests/test_environment.py`

In [3]:
# use ! to run shell commands in notebook
! nosetests tests/test_environment.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [4]:
import mynlplib.parsing as parsing
import mynlplib.data_tools as data_tools
import mynlplib.constants as consts
import mynlplib.evaluation as evaluation
import mynlplib.utils as utils
import mynlplib.feat_extractors as feat_extractors
import mynlplib.neural_net as neural_net

In [5]:
# Read in the datasets
reload(data_tools)
en_dataset = data_tools.Dataset(consts.EN_TRAIN_FILE, consts.EN_DEV_FILE, consts.EN_TEST_FILE)
nr_dataset = data_tools.Dataset(consts.NR_TRAIN_FILE, consts.NR_DEV_FILE, consts.NR_TEST_FILE)

# Assign each word a unique index, including two special tokens needed for parsing logic
word_to_ix_en = { word: i for i, word in enumerate(en_dataset.vocab) }
word_to_ix_nr = { word: i for i, word in enumerate(nr_dataset.vocab) }

In [6]:
# Some constants to keep around
LSTM_NUM_LAYERS = 1
TEST_EMBEDDING_DIM = 4
WORD_EMBEDDING_DIM = 64
STACK_EMBEDDING_DIM = 100
NUM_FEATURES = 3

# Hyperparameters
ETA_0 = 0.01
DROPOUT = 0.0

# High-Level Overview of the Parser
Be sure that you have reviewed Eisenstein Ch. 11.3 on transition-based dependency parsing, and are familiar with the relevant terminology.
Parsing will proceed as follows:
* Initialize your parsing stack and input buffer.
* At each step, until the parse is done:
  * Extract some features.  We will start with simple features, but these can be anything: words in the sentence, the configuration of the stack, the configuration of the input buffer, the previous action, etc.
  * Send these features through a feed-forward (FF) network to get a probability distribution over actions (`SHIFT`, `ARC_L`, `ARC_R`).  The next action you choose is the one with the highest probability.
  * If the action is an arc- operation, you use a neural network to combine the two items in the operation and get a dense output to place back on the input buffer.

The key classes you will fill in code for are:
* Feature extraction in `feat_extractors.py`
* The `ParserState` class, which keeps track of the input buffer and parse stack, and offers a public interface for doing the parsing actions to update the state
* The `TransitionParser` class, which is a PyTorch module where the core parsing logic resides, in `parsing.py`.
* The neural network components in `neural_net.py`

The network components are compartmentalized as follows:
* `TransitionParser`, the base component that contains and coordinates the other substitutable components

* Embedding Lookup: You will implement three flavors of embeddings. These embeddings are used to initialize the input buffer, and will be shifted on the stack / serve as inputs to the combiner networks.
  - `VanillaWordEmbedding` just gets embeddings from a lookup table.
  - `BiLSTMWordEmbedding` will run a sequence model in both directions over the sentence. The hidden state at step t is the embedding for the `t`-th word of the sentence.
  - `SuffixAndWordEmbedding` gets embeddings for words as in the vanilla embeddings, and also gets embeddings for word suffixes, and concatenates them together.
* Action Choosing: You will implement two action choosing components:
  - `FFActionChooser` is a simple feed-forward neural network that outputs log probabilities over the three actions given the extracted features as input.
  - `LSTMActionChooser` applies a sequence model that takes the hidden state of the previous action decision as input.

* Combiners: You will implement two combiners, which are the network components that take the two embeddings of the items in an arc- operation and creates a single vector.
  - `FFCombiner` takes the two input embeddings and gives a dense output.
  - `LSTMCombiner` applies a sequence model, where the output embedding is the hidden state of the next timestep.

### Parsing example

The following is how the input buffer and stack look at each step of a parse, up to the first arc.  The input sentence is "the dog ran away".  Our action chooser network takes the top element of the stack, the top element of the input buffer, plus a one-token "lookahead" in the input buffer.  $C(x,y)$ refers to calling our combiner network on arguments $x, y$.  Also let $A$ be the set of actions: $\{ \text{SHIFT}, \text{ARC-L}, \text{ARC-R} \}$, and let $q_w$ be the embedding for word $w$.

1. 
  * Input Buffer: $\left[ q_\text{the}, q_\text{dog}, q_\text{ran}, q_\text{away}, q_\text{END-INPUT} \right]$
  * Stack: $\left[ q_\text{ROOT} \right]$
  * Action: $ \text{argmax}_{a \in A} \ \text{ActionChooser}(q_\text{ROOT}, q_\text{the}, \overbrace{q_\text{dog}}^\text{lookahead}) \Rightarrow \text{SHIFT}$
  
2.
  * Input Buffer: $\left[ q_\text{dog}, q_\text{ran}, q_\text{away}, q_\text{END-INPUT} \right]$
  * Stack: $\left[ q_\text{ROOT}, q_\text{the} \right]$
  * Action: $ \text{argmax}_{a \in A} \ \text{ActionChooser}(q_\text{the}, q_\text{dog}, q_\text{ran}) \Rightarrow \text{ARC-L}$
  
3.
  * Input Buffer: $\left[C(q_\text{dog}, q_\text{the}), q_\text{ran}, q_\text{away}, q_\text{END-INPUT} \right]$
  * Stack: $\left[ q_\text{ROOT} \right]$
  
This is a partial picture of parsing - we keep more than just the embedding on the stack and input buffer.  We also keep the word and its position in the sentence so that when we create an arc, we know what edge was just created.
So, for example, the initial input buffer really looks like

$$ \left[ (\text{the}, 0, q_\text{the}), (\text{dog}, 1, q_\text{dog}), (\text{ran}, 2, q_\text{ran}), (\text{away}, 3, q_\text{away}), (\text{END-INPUT}, 4, q_\text{END-INPUT}) \right] $$

Before beginning, I recommend completing the parse by hand, drawing the input buffer and stack at each step, and explicity listing the arguments to the action chooser.

# 1. Managing and Updating the Parser State (12 points)

In this part of the assignment, you will work with the ParserState class, that keeps track of the parser's input buffer and stack.

### Deliverable 1.1: Implementing Arc

#### 1.1a: Get arc components (2 points)
You will implement the generalized arc- operation of the `ParserState` in `parsing.py`, in the function `_arc`, in two parts.

First, fill in `_get_arc_components` in `parsing.py` to select the head and modifier according to the action passed in. This method should also remove the items from the stack and input buffer.
Arc actions follow the arc-standard procedure, from Eisenstein Ch. 11.3.1.

- **Test**: ` test_parser.py:test_get_arc_components_d1_1a`

In [7]:
reload(parsing)
test_sentence = "The man ran away".split()
parser_state = parsing.ParserState(test_sentence + [consts.END_OF_INPUT_TOK], 
                                   [None] * (len(test_sentence)+1),
                                   utils.DummyCombiner())

In [8]:
parser_state.shift()
parser_state.shift()
print(parser_state)

head, modifier = parser_state._get_arc_components(consts.Actions.ARC_L)
print(head, modifier)

head, modifier = parser_state._get_arc_components(consts.Actions.ARC_R)
print(head, modifier)

Stack: ['<ROOT>', 'The', 'man']
Input Buffer: ['ran', 'away', '<END-OF-INPUT>']

StackEntry(headword='ran', headword_pos=2, embedding=None) StackEntry(headword='man', headword_pos=1, embedding=None)
StackEntry(headword='The', headword_pos=0, embedding=None) StackEntry(headword='away', headword_pos=3, embedding=None)


#### 1.1b: Create the arc (2 points)
Now, fill in `_create_arc` in `parsing.py` to use the `ParserState`'s `combiner` component to **combine** the passed in head and modifier, put the combination on the input buffer, and create a new dependency graph edge. At this point, we are just using a dummy combiner so we can test the logic.

You will want to familiarize yourself with the `StackEntry` and `DepGraphEdge` objects used by the `ParserState` object for this one.

- **Test**: ` test_parser.py:test_create_arc_d1_1b`

In [9]:
reload(parsing)
parser_state = parsing.ParserState(test_sentence + [consts.END_OF_INPUT_TOK], 
                                   [None] * (len(test_sentence)+1),
                                   utils.DummyCombiner())

print(parser_state)

parser_state.shift()
print(parser_state)

arc = parser_state.arc_left()
print("First arc: Head: {}, Modifier: {}".format(arc[0], arc[1]), "\n")
print(parser_state)

parser_state.shift()
arc = parser_state.arc_left()
print("Second arc: Head: {}, Modifier: {}".format(arc[0], arc[1]), "\n")
print(parser_state)

Stack: ['<ROOT>']
Input Buffer: ['The', 'man', 'ran', 'away', '<END-OF-INPUT>']

Stack: ['<ROOT>', 'The']
Input Buffer: ['man', 'ran', 'away', '<END-OF-INPUT>']

First arc: Head: ('man', 1), Modifier: ('The', 0) 

Stack: ['<ROOT>']
Input Buffer: ['man', 'ran', 'away', '<END-OF-INPUT>']

Second arc: Head: ('ran', 2), Modifier: ('man', 1) 

Stack: ['<ROOT>']
Input Buffer: ['ran', 'away', '<END-OF-INPUT>']



### Deliverable 1.2: Parser Terminating Condition (4 points)
In this short (one line) deliverable, implement `done_parsing()` in `ParserState`.  Think about what the input buffer and stack look like at the end of a parse.

- **Test**: `test_parsing.py:test_stack_terminating_cond_d1_2`

In [10]:
reload(parsing)
parser_state = parsing.ParserState(test_sentence + [consts.END_OF_INPUT_TOK], 
                                   [None] * (len(test_sentence)+1),
                                   utils.DummyCombiner())

parser_state.shift()
parser_state.arc_left()
parser_state.shift()
parser_state.arc_left()

print(parser_state.done_parsing())

parser_state.shift()
parser_state.arc_right()
print(parser_state.done_parsing())

parser_state.arc_right()
print(parser_state.done_parsing())

parser_state.shift()
print(parser_state.done_parsing())

False
False
False
True


### Deliverable 1.3: Validating parser actions (4 points)
Implement the `_validate_action` method in `parsing.TransitionParser`. This will be used in the prediction setting, when the gold standard is not available. We need to ensure that any action we take is legal. Here are the rules:

- You cannot shift when the input buffer has <= 2 items on it (including the end of input token), UNLESS the stack is empty.
  - **In this case, do `ARC_R` by default.**
- You cannot do an arc- operation when the stack is empty (this will happen after creating an arc with ROOT).
  - **In this case, do `SHIFT` by default.**
- You cannot do an arc-left operation when the root token is on top of the stack.
  - **In this case, do `SHIFT` or `ARC-R` depending on the state of the input buffer.**
  
**Test:**
- `test_parser.py:test_validate_action_d1_3`

**Make sure you pass the test before you move on. The code blocks below are not meant to be comprehensive.**

In [11]:
reload(parsing)
parser_state = parsing.ParserState(test_sentence + [consts.END_OF_INPUT_TOK], 
                                   [None] * (len(test_sentence)+1),
                                   utils.DummyCombiner())
ix_to_action = consts.Actions.ix_to_action

In [12]:
print(parser_state)
act_to_do = consts.Actions.ARC_L
valid_action = parser_state._validate_action(act_to_do)
print("Chosen action: %s, Valid action: %s\n" % (ix_to_action[act_to_do], ix_to_action[valid_action]))

parser_state.shift()

print(parser_state)
act_to_do = consts.Actions.ARC_L
valid_action = parser_state._validate_action(act_to_do)
print("Chosen action: %s, Valid action: %s\n" % (ix_to_action[act_to_do], ix_to_action[valid_action]))

parser_state.shift()
parser_state.shift()

print(parser_state)
act_to_do = consts.Actions.SHIFT
valid_action = parser_state._validate_action(act_to_do)
print("Chosen action: %s, Valid action: %s\n" % (ix_to_action[act_to_do], ix_to_action[valid_action]))

Stack: ['<ROOT>']
Input Buffer: ['The', 'man', 'ran', 'away', '<END-OF-INPUT>']

Chosen action: ARC_L, Valid action: SHIFT

Stack: ['<ROOT>', 'The']
Input Buffer: ['man', 'ran', 'away', '<END-OF-INPUT>']

Chosen action: ARC_L, Valid action: ARC_L

Stack: ['<ROOT>', 'The', 'man', 'ran']
Input Buffer: ['away', '<END-OF-INPUT>']

Chosen action: SHIFT, Valid action: ARC_R



# 2. Neural Network for Action Decisions (12 points)
In this part of the assignment, you will use PyTorch to create a neural network which examines the current state of the parse and makes the decision to either shift, arc left, or arc right.

### Deliverable 2.1: Word Embedding Lookup (3 points)
Implement the class `VanillaWordEmbedding` in `neural_net.py`
([Docs for Pytorch embeddings](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html))

Hint: You will have to turn the input, which is a list of strings (the words in the sentence), into a format that your embedding lookup table can take. 

**Test:** `test_parser.py:test_word_embed_lookup_d2_1`

In [13]:
reload(neural_net)
torch.manual_seed(1)

test_sentence = "natural language processing".split()
test_word_to_ix = { "natural": 0, "language": 1, "processing": 2 }

word_embedder = neural_net.VanillaWordEmbedding(test_word_to_ix, TEST_EMBEDDING_DIM)
embeds = word_embedder(test_sentence)
print(type(embeds))
print(len(embeds), "\n")
print("Embedding for 'natural':\n {}".format(embeds[0]))

<class 'list'>
3 

Embedding for 'natural':
 tensor([[0.6614, 0.2669, 0.0617, 0.6213]], grad_fn=<ViewBackward>)


### Deliverable 2.2: Feature Extraction (3 points)
Fill in the `SimpleFeatureExtractor` class in `feat_extractors.py` to give the following 3 features as a list **in this order**:
* The embedding of the top of the stack
* The embedding of the first token in the input buffer
* The embedding of the next token in the input buffer (one-token lookahead)

If at this point you have not poked around `ParserState` to see how it stores the state, now would be a good time.

**Test:** `test_parser.py:test_feature_extraction_d2_2`

In [14]:
reload(feat_extractors)
torch.manual_seed(1)

test_sentence = "The Sound and the Fury".split()
test_word_to_ix = { word: i for i, word in enumerate(sorted(set(test_sentence))) }

embedder = neural_net.VanillaWordEmbedding(test_word_to_ix, TEST_EMBEDDING_DIM)
embeds = embedder(test_sentence)

state = parsing.ParserState(test_sentence, embeds, utils.DummyCombiner())

state.shift()
feat_extractor = feat_extractors.SimpleFeatureExtractor()
feats = feat_extractor.get_features(state)

print("Embedding for 'The':\n {}".format(feats[0]))
print("Embedding for 'Sound':\n {}".format(feats[1]))
print("Embedding for 'and' (from buffer lookahead):\n {}".format(feats[2]))

Embedding for 'The':
 tensor([[ 0.4391,  1.1712,  1.7674, -0.0954]], grad_fn=<ViewBackward>)
Embedding for 'Sound':
 tensor([[ 0.8657,  0.2444, -0.6629,  0.8073]], grad_fn=<ViewBackward>)
Embedding for 'and' (from buffer lookahead):
 tensor([[ 0.0612, -0.6177, -0.7981, -0.1316]], grad_fn=<ViewBackward>)


### Deliverable 2.3: Feedforward Network for Choosing Actions (3 points)
Implement the class `neural_net.FFActionChooser` according to the specification.
You will need to take the list of embeddings passed in (that come from your feature extractor) and concatenate them to one long row vector (size [1 x num actions])

This network takes as input the features from your feature extractor, concatenates them, runs them through a feedforward network, and outputs log probabilities over actions.

**Test:** `test_parser.py:test_action_chooser_d2_3`

In [15]:
reload(neural_net)
torch.manual_seed(1)
act_chooser = neural_net.FFActionChooser(TEST_EMBEDDING_DIM * NUM_FEATURES)
feats = [ ag.Variable(torch.randn(1, TEST_EMBEDDING_DIM)) for _ in range(NUM_FEATURES) ] # make some dummy feature embeddings
log_probs = act_chooser(feats)
print(log_probs)

tensor([[-1.2443, -0.8323, -1.2844]], grad_fn=<LogSoftmaxBackward>)


### Deliverable 2.4: Network for Combining Stack Items (3 points)
Implement the class `neural_net.FFCombiner` according to the specification. 
Recall that what this component does is take two embeddings, the head and modifier, during an arc- operation and output a combined embedding (of size [1 x embedding_dim]), which is then pushed back onto the input buffer during parsing.

**Test:** `test_parser.py:test_combiner_d2_4`

In [16]:
reload(neural_net)
torch.manual_seed(1)
combiner = neural_net.FFCombiner(TEST_EMBEDDING_DIM)

# Again, make dummy inputs
head_feat = ag.Variable(torch.randn(1, TEST_EMBEDDING_DIM))
modifier_feat = ag.Variable(torch.randn(1, TEST_EMBEDDING_DIM))
combined = combiner(head_feat, modifier_feat)
print(combined)

tensor([[ 0.4285, -0.1363,  0.4046,  0.6006]], grad_fn=<AddmmBackward>)


# 3. Return of the Parser (12 points)

### Deliverable 3.1: Parser Training Code (8 points)
We will now complete the parser and train it on our data. It is important to understand the difference between the following tasks:

* Training: Training the model involves passing it sentences along with the correct sequence of actions, and updating weights.
* Evaluation: We can evaluate the parser by passing it sentences along with the correct sequence of actions, and see how many actions it predicts correctly.  This is identical to training, except the weights are not updated after making a prediction.
* Prediction: After setting the weights, we give it a raw sentence (no gold-standard actions), and let it follow its own predicted actions to create a dependency graph, which we can compare to the ground truth.

You will implement the `forward()` function in `mynlplib.parsing.TransitionParser`.

At this point, it is necessary to have all of the components from part 2 in place for constructing the parser.

The parsing logic is roughly as follows:
* Loop until parsing state is in its terminating state (deliverable 1.2)
* Get the features from the parsing state (deliverable 2.2)
* Send them through your action chooser network to get log probabilities over actions (deliverable 2.3)
* If you have `gold_actions`, do them. Otherwise (when predicting), take the argmax of your log probabilities, validate the action (deliverable 1.3), and do that. An argmax function is provided for you in `utils.argmax`.

Make sure to keep track of the things that the function wants to keep track of
* Do all of your actions by calling the appropriate function on your `parser_state`
* Append each output autograd.Variable from your action_chooser to the outputs list
* Append each action you do to `actions_done`
* Build the set of dependency edges as you go

**Tests:**
- `test_parser.py:test_parse_logic_d3_1`
- `test_parser.py:test_predict_after_train_d3_1`

In [17]:
test_sentence = "The man ran away".split()
test_word_to_ix = { word: i for i, word in enumerate(sorted(set(test_sentence))) }
test_word_to_ix[consts.END_OF_INPUT_TOK] = len(test_word_to_ix)
test_sentence_vocab = set(test_sentence)
gold_actions = ["SHIFT", "ARC_L", "SHIFT", "ARC_L", "SHIFT", "ARC_R", "ARC_R", "SHIFT"]

In [18]:
reload(parsing)
torch.manual_seed(1)
feat_extractor = feat_extractors.SimpleFeatureExtractor()
word_embedding_lookup = neural_net.VanillaWordEmbedding(test_word_to_ix, STACK_EMBEDDING_DIM)
action_chooser = neural_net.FFActionChooser(STACK_EMBEDDING_DIM * NUM_FEATURES)
combiner_network = neural_net.FFCombiner(STACK_EMBEDDING_DIM)
parser = parsing.TransitionParser(feat_extractor, word_embedding_lookup,
                                     action_chooser, combiner_network)
output, depgraph, actions_done = parser(test_sentence, gold_actions)
print(depgraph)
print(actions_done)

{DepGraphEdge(head=('<ROOT>', -1), modifier=('ran', 2)), DepGraphEdge(head=('ran', 2), modifier=('man', 1)), DepGraphEdge(head=('man', 1), modifier=('The', 0)), DepGraphEdge(head=('ran', 2), modifier=('away', 3))}
[0, 1, 0, 1, 0, 2, 2, 0]


### Now Train the Parser!

Training your parser may take some time. There are 10,000 training sentences. We take a subset of them here.

In [19]:
def train_parser(parser, optimizer, dataset, n_epochs=1, n_train_insts=1000):
    for epoch in range(n_epochs):
        print("Epoch {}".format(epoch+1))

        parser.train() # turn on dropout layers if they are there
        parsing.train(dataset.training_data[:n_train_insts], parser, optimizer, verbose=True)

        print("Dev Evaluation")
        parser.eval() # turn them off for evaluation
        parsing.evaluate(dataset.dev_data, parser, verbose=True)
        print("F-Score: {}".format(evaluation.compute_metric(parser, dataset.dev_data, evaluation.fscore)))
        print("Attachment Score: {}".format(evaluation.compute_attachment(parser, dataset.dev_data)))
        print("\n")

In [20]:
reload(parsing)
torch.manual_seed(1)
feat_extractor = feat_extractors.SimpleFeatureExtractor()
word_embedding_lookup = neural_net.VanillaWordEmbedding(word_to_ix_en, STACK_EMBEDDING_DIM)
action_chooser = neural_net.FFActionChooser(STACK_EMBEDDING_DIM * NUM_FEATURES)
combiner_network = neural_net.FFCombiner(STACK_EMBEDDING_DIM)
parser = parsing.TransitionParser(feat_extractor, word_embedding_lookup,
                                     action_chooser, combiner_network)
optimizer = optim.SGD(parser.parameters(), lr=ETA_0)

In [21]:
%%timeit
torch.manual_seed(1)
parsing.train(en_dataset.training_data[:100], parser, optimizer, verbose=True)

Number of instances: 100    Number of network actions: 4836
Acc: 0.7078163771712159  Loss: 32.37324492931366
Number of instances: 100    Number of network actions: 4836
Acc: 0.8438792390405294  Loss: 18.007621763944627
Number of instances: 100    Number of network actions: 4836
Acc: 0.9011579818031431  Loss: 12.509024324715137
Number of instances: 100    Number of network actions: 4836
Acc: 0.934449958643507  Loss: 7.615389933176338
Number of instances: 100    Number of network actions: 4836
Acc: 0.9611248966087675  Loss: 5.287574468702078
Number of instances: 100    Number of network actions: 4836
Acc: 0.9739454094292804  Loss: 3.574098753067665
Number of instances: 100    Number of network actions: 4836
Acc: 0.9793217535153019  Loss: 2.704069236360956
Number of instances: 100    Number of network actions: 4836
Acc: 0.989454094292804  Loss: 1.5278681585029699
1.79 s ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
# train the parser for a while here.
# Shouldn't take *too* long, even on a laptop
torch.manual_seed(1)
train_parser(parser, optimizer, en_dataset, n_train_insts=1000)

Epoch 1
Number of instances: 1000    Number of network actions: 44560
Acc: 0.8259425493716338  Loss: 20.41987961708894
Dev Evaluation
Number of instances: 501    Number of network actions: 15846
Acc: 0.8233623627413859  Loss: 15.728462156808067
F-Score: 0.464984839911928
Attachment Score: 0.43910135049854854




### Deliverable 3.2: Test Data Predictions (2 points)
Run the code below to output your predictions on the test data and dev data.  You can run the dev test to verify you are correct up to this point.  The test data evaluation is for us.

**Test**: `test_parser.py:test_dev_d3_2_english`

In [23]:
dev_sentences = [ sentence for sentence, _ in en_dataset.dev_data ]
evaluation.output_preds(consts.EN_D3_2_DEV_FILENAME, parser, dev_sentences)

In [24]:
evaluation.output_preds(consts.EN_D3_2_TEST_FILENAME, parser, en_dataset.test_data)

### Deliverable 3.3: Dependency parsing in Norwegian (2 points)
Run the code below to output your predictions on the **norwegian** test data and dev data.  You can run the dev test to verify you are correct up to this point.  The test data evaluation is for us.

**Test**: `test_parser.py:test_dev_d3_3_norwegian`

In [25]:
reload(parsing)
torch.manual_seed(1)
feat_extractor_nr = feat_extractors.SimpleFeatureExtractor()
word_embedding_lookup_nr = neural_net.VanillaWordEmbedding(word_to_ix_nr, STACK_EMBEDDING_DIM)
action_chooser_nr = neural_net.FFActionChooser(STACK_EMBEDDING_DIM * NUM_FEATURES)
combiner_network_nr = neural_net.FFCombiner(STACK_EMBEDDING_DIM)
parser_nr = parsing.TransitionParser(feat_extractor_nr, word_embedding_lookup_nr,
                                     action_chooser_nr, combiner_network_nr)
optimizer_nr = optim.SGD(parser_nr.parameters(), lr=ETA_0)

In [26]:
torch.manual_seed(1)
train_parser(parser_nr, optimizer_nr, nr_dataset, n_epochs=2, n_train_insts=1000)

Epoch 1
Number of instances: 1000    Number of network actions: 30942
Acc: 0.8113890504815461  Loss: 13.949815350351855
Dev Evaluation
Number of instances: 501    Number of network actions: 16028
Acc: 0.8226229099076616  Loss: 13.585948317384203
F-Score: 0.4521501957801757
Attachment Score: 0.4361118043424008


Epoch 2
Number of instances: 1000    Number of network actions: 30942
Acc: 0.8785792773576369  Loss: 9.363433301108715
Dev Evaluation
Number of instances: 501    Number of network actions: 16028
Acc: 0.8264287496880459  Loss: 15.430829859188417
F-Score: 0.4660331380773358
Attachment Score: 0.4472173696031944




In [27]:
reload(evaluation)
dev_sentences_nr = [ sentence for sentence, _ in nr_dataset.dev_data ]
evaluation.output_preds(consts.NR_D3_3_DEV_FILENAME, parser_nr, dev_sentences_nr)

In [28]:
evaluation.output_preds(consts.NR_D3_3_TEST_FILENAME, parser_nr, nr_dataset.test_data)

# 4. Evaluation and Training Improvements (18 points)

### Deliverable 4.1: BiLSTM Word Embeddings (3 points)
Implement the class `BiLSTMWordEmbedding` in `neural_net.py`.
This class can replace your `VanillaWordEmbedding`.
This class implements a sequence model over the sentence, where the t'th word's embedding is the hidden state at timestep t.
This means that, rather than have our embeddings on the stack only include the semantics of a single word, our embeddings will contain information from all parts of the sentence (the LSTM will, in principle, learn what information is relevant).

**Test**: `tests/test_parser.py:test_bilstm_word_embeds_d4_1`

In [29]:
reload(neural_net)
torch.manual_seed(1)
test_sentence = "Noam Chomsky".split()
test_word_to_ix = { "Noam": 0, "Chomsky": 1 }

lstm_word_embedder = neural_net.BiLSTMWordEmbedding(test_word_to_ix,
                                                    WORD_EMBEDDING_DIM,
                                                    STACK_EMBEDDING_DIM,
                                                    num_layers=LSTM_NUM_LAYERS,
                                                    dropout=DROPOUT)
    
lstm_embeds = lstm_word_embedder(test_sentence)
print(type(lstm_embeds))
print(len(lstm_embeds), "\n")
print("Embedding for Noam:\n {}".format(lstm_embeds[0]))

<class 'list'>
2 

Embedding for Noam:
 tensor([[-6.3007e-02,  2.4330e-01, -7.0760e-02, -1.1852e-01,  1.8881e-01,
         -1.6543e-01, -2.1600e-02,  1.4040e-02, -6.8058e-02, -1.8666e-01,
          1.0207e-01,  2.2894e-02, -5.8540e-02, -6.3337e-02, -2.9607e-01,
         -2.0053e-02, -1.8389e-01, -9.1271e-02, -5.1386e-02, -3.4879e-01,
         -3.8826e-02,  8.8795e-02, -3.8836e-02,  1.2170e-02,  4.6013e-02,
         -1.3923e-01,  1.9091e-02,  7.1751e-02,  9.5653e-02, -3.5629e-01,
          1.9788e-01,  2.9786e-02,  6.1633e-02,  4.7286e-02, -2.9223e-01,
         -7.4602e-02,  2.4812e-01, -1.3309e-01,  4.2635e-02,  4.2023e-02,
          3.1180e-02,  5.5482e-03, -1.1297e-01,  1.4215e-02, -1.0769e-01,
         -1.4725e-01, -7.3080e-02,  2.1588e-02,  1.7645e-01,  4.3659e-02,
         -2.4070e-04,  1.1204e-02, -2.2866e-01,  1.1086e-01, -3.3928e-02,
         -1.3846e-01, -8.5202e-03,  8.6117e-02,  9.5097e-02, -1.2923e-01,
         -2.7905e-03, -6.9797e-02,  1.6902e-01, -1.0969e-01, -1.3452e-01

### Deliverable 4.2: Suffix Embeddings (3 points)
We can also try to more explicitly include morphological information by embedding the suffix of a word in addition to the word itself. We approximate the "suffix" by just looking at the last two characters of a word.

First, implement the function `build_suff_to_ix` in `utils.py`. It should take in a `word_to_ix` lookup and return a `suff_to_ix` lookup.

Then, implement the class `SuffixAndWordEmbedding` in `neural_net.py`.
This class embeds the words and suffixes in a sentence and then concatenates them to form one embedding. 

**Test**: `tests/test_parser.py:test_suff_word_embeds_d4_2`

In [30]:
reload(utils)
suff_to_ix_en = utils.build_suff_to_ix(word_to_ix_en)
suff_to_ix_nr = utils.build_suff_to_ix(word_to_ix_nr)

In [31]:
len(suff_to_ix_en), len(suff_to_ix_nr)

(1145, 849)

In [32]:
reload(neural_net)
torch.manual_seed(1)
test_sentence = "prefix fixsuf fixinfix".split()
test_word_to_ix = { "prefix": 0, "fixsuf": 1, "fixinfix": 2 }
test_suff_to_ix = utils.build_suff_to_ix(test_word_to_ix)

suff_word_embedder = neural_net.SuffixAndWordEmbedding(test_word_to_ix, test_suff_to_ix, TEST_EMBEDDING_DIM)
test_embs = suff_word_embedder(test_sentence)

In [33]:
test_embs[0]

tensor([[ 0.6614,  0.2669, -1.5228,  0.3817]], grad_fn=<ViewBackward>)

### Deliverable 4.3: Pretrained Embeddings (2 points)

Fill in the function `initialize_with_pretrained` in `utils.py`.

It will take a word embedding lookup component and initialize its lookup table with pretrained embeddings, which are provided. Note that this is only applicable for the Vanilla, BiLSTM, and SuffixAndWord embedding components.

**Test**: `tests/test_parser.py:test_pretrained_embeddings_d4_3`

In [34]:
import pickle
pretrained_embeds = pickle.load(open(consts.PRETRAINED_EMBEDS_FILE, 'rb'))
print(pretrained_embeds['four'][:5])

[0.12429751455783844, -0.11472601443529129, -0.5684014558792114, -0.396965891122818, 0.22938089072704315]


In [35]:
torch.manual_seed(1)
embedder = neural_net.VanillaWordEmbedding(word_to_ix_en,64)

In [36]:
embedder.forward(['four'])[0][0,:5]

tensor([-0.5776,  0.2705,  0.4988,  0.1272,  0.6237], grad_fn=<SliceBackward>)

In [37]:
reload(utils);
utils.initialize_with_pretrained(pretrained_embeds,embedder)
print(embedder.forward(['four'])[0][0,:5])

tensor([ 0.1243, -0.1147, -0.5684, -0.3970,  0.2294], grad_fn=<SliceBackward>)


### Deliverable 4.4: Better Arc Component Combination (3 points)
Before, in order to combine two embeddings during an arc- operation, we just passed them through a feed-forward network and got a dense output.  Now, we will instead use a sequence model of the stack.  The combined embedding from an arc- operation is the next time step of an LSTM.  Implement `neural_net.LSTMCombiner`.

**Test**: `tests/test_parser.py:test_lstm_combiner_d4_4`

In [38]:
reload(neural_net)
torch.manual_seed(1)
combiner = neural_net.LSTMCombiner(TEST_EMBEDDING_DIM,
                                          num_layers=LSTM_NUM_LAYERS,
                                          dropout=DROPOUT)
head_feat = ag.Variable(torch.randn(1,TEST_EMBEDDING_DIM))
mod_feat = ag.Variable(torch.randn(1,TEST_EMBEDDING_DIM))

In [39]:
combined = combiner(head_feat, mod_feat)
combined

tensor([[ 0.0532, -0.1534,  0.1484, -0.0595]], grad_fn=<SelectBackward>)

### Deliverable 4.5: Better action choosing (3 points)
Instead of choosing the action from the combiner output independently at each time step, let's use an LSTM to predict the action. This way, past actions can influence the current decision directly. 

Implement `neural_net.LSTMActionChooser`. Use a linear layer to predict the action from the LSTM hidden state.

**Test**: `tests/test_parser.py:test_lstm_action_chooser_d4_5`

In [40]:
reload(neural_net)
torch.manual_seed(1)
action_chooser = neural_net.LSTMActionChooser(TEST_EMBEDDING_DIM * NUM_FEATURES,
                                                     LSTM_NUM_LAYERS,
                                                     dropout=DROPOUT)
feats = [ag.Variable(torch.randn(1, TEST_EMBEDDING_DIM)) for _ in range(NUM_FEATURES)]

In [41]:
output = action_chooser(feats)
output

tensor([[-1.0328, -1.1798, -1.0887]], grad_fn=<LogSoftmaxBackward>)

### Retrain with the new components

In [42]:
reload(utils)
reload(neural_net)
reload(parsing)
reload(feat_extractors)
torch.manual_seed(1)
stack_dim = STACK_EMBEDDING_DIM
feat_extractor = feat_extractors.SimpleFeatureExtractor()
# BiLSTM word embeddings will probably work best, but feel free to experiment with the others you developed
word_embedding_lookup = neural_net.BiLSTMWordEmbedding(word_to_ix_en,
                                                       WORD_EMBEDDING_DIM,
                                                       STACK_EMBEDDING_DIM,
                                                       num_layers=LSTM_NUM_LAYERS,
                                                       dropout=DROPOUT)
utils.initialize_with_pretrained(pretrained_embeds, word_embedding_lookup)
action_chooser = neural_net.LSTMActionChooser(STACK_EMBEDDING_DIM * NUM_FEATURES,
                                              LSTM_NUM_LAYERS,
                                              dropout=DROPOUT)
combiner = neural_net.LSTMCombiner(STACK_EMBEDDING_DIM,
                                   num_layers=LSTM_NUM_LAYERS,
                                   dropout=DROPOUT)
parser = parsing.TransitionParser(feat_extractor, word_embedding_lookup,
                                  action_chooser, combiner)
optimizer = optim.SGD(parser.parameters(), lr=ETA_0)

In [43]:
# The LSTMs will make this take longer, probably just a few minutes
train_parser(parser, optimizer, en_dataset, n_epochs=2, n_train_insts=1000)

Epoch 1
Number of instances: 1000    Number of network actions: 44560
Acc: 0.796229802513465  Loss: 20.93961408064142
Dev Evaluation
Number of instances: 501    Number of network actions: 15846
Acc: 0.8564937523665278  Loss: 10.963262027847076
F-Score: 0.584904339949434
Attachment Score: 0.5598889309604947


Epoch 2
Number of instances: 1000    Number of network actions: 44560
Acc: 0.8833034111310593  Loss: 12.569501896796748
Dev Evaluation
Number of instances: 501    Number of network actions: 15846
Acc: 0.88034835289663  Loss: 9.406927095454728
F-Score: 0.6343035073922545
Attachment Score: 0.6134040136312003




### Deliverable 4.6: Test Predictions: English (2 point)

**Test**: `tests/test_parser.py:test_dev_preds_d4_6_english`

In [44]:
dev_sentences = [ sentence for sentence, _ in en_dataset.dev_data ]
evaluation.output_preds(consts.EN_D4_6_DEV_FILENAME, parser, dev_sentences)

In [45]:
evaluation.output_preds(consts.EN_D4_6_TEST_FILENAME, parser, en_dataset.test_data)

### Deliverable 4.7: Test Predictions: Norwegian (2 point)

**Test**: `tests/test_parser.py:test_dev_preds_d4_7_norwegian`

In [46]:
torch.manual_seed(1)
feat_extractor_nr = feat_extractors.SimpleFeatureExtractor()
# BiLSTM word embeddings will probably work best, but feel free to experiment with the others you developed
word_embedding_lookup_nr = neural_net.BiLSTMWordEmbedding(word_to_ix_nr,
                                                          WORD_EMBEDDING_DIM,
                                                          STACK_EMBEDDING_DIM,
                                                          num_layers=LSTM_NUM_LAYERS,
                                                          dropout=DROPOUT)
action_chooser_nr = neural_net.FFActionChooser(STACK_EMBEDDING_DIM * NUM_FEATURES)
combiner_nr = neural_net.LSTMCombiner(STACK_EMBEDDING_DIM,
                                          num_layers=LSTM_NUM_LAYERS,
                                          dropout=DROPOUT)
parser_nr = parsing.TransitionParser(feat_extractor_nr, word_embedding_lookup_nr,
                                  action_chooser_nr, combiner_nr)
optimizer_nr = optim.SGD(parser_nr.parameters(), lr=ETA_0)

In [47]:
train_parser(parser_nr, optimizer_nr, nr_dataset, n_epochs=3, n_train_insts=1000)

Epoch 1
Number of instances: 1000    Number of network actions: 30942
Acc: 0.8142653997802339  Loss: 13.2001195525378
Dev Evaluation
Number of instances: 501    Number of network actions: 16028
Acc: 0.8396556026952833  Loss: 11.748504309730972
F-Score: 0.5122631285979322
Attachment Score: 0.48290491639630645


Epoch 2
Number of instances: 1000    Number of network actions: 30942
Acc: 0.8847197983323638  Loss: 8.464239425085484
Dev Evaluation
Number of instances: 501    Number of network actions: 16028
Acc: 0.8597454454704268  Loss: 10.622585836366682
F-Score: 0.5689138768007578
Attachment Score: 0.540678812078862


Epoch 3
Number of instances: 1000    Number of network actions: 30942
Acc: 0.9161657294292548  Loss: 6.277243318439927
Dev Evaluation
Number of instances: 501    Number of network actions: 16028
Acc: 0.8652358372847517  Loss: 10.927354865651225
F-Score: 0.5818803188918262
Attachment Score: 0.5481657100074869




In [48]:
dev_sentences_nr = [ sentence for sentence, _ in nr_dataset.dev_data ]
evaluation.output_preds(consts.NR_D4_7_DEV_FILENAME, parser_nr, dev_sentences_nr)

In [49]:
evaluation.output_preds(consts.NR_D4_7_TEST_FILENAME, parser_nr, nr_dataset.test_data)

# 5. Bakeoff (24 points)

We will have another bakeoff for this problem set.

Try to implement new features and tune your network's architecture and hyperparameters to get the best network.
Section 3 of [this paper](https://arxiv.org/pdf/1206.5533.pdf) may help out with hyper parameter tuning if you are new to neural networks.
To get very competitive, it may be necessary to train for a large amount of time (leaving it running overnight should be fine).  Here are some suggestions.
* Tune your learning rate.
* Tune your other hyperparameters.
* Try different optimizers.  torch.optim has a ton of different training algorithms.  SGD was used in this pset because it is fast, but it is the most vanilla of them.  Trying new ones, like Adam, will almost certainly boost performance
* Try adding regularization to your network if you see evidence that it is overfitting. This can be done with:
  * L2 regularization using the [weight decay argument](http://pytorch.org/docs/master/optim.html#torch.optim.SGD)
  * adding dropout (already an input argument to some of the neural net components)
  * implement early stopping (stop training if dev performance on some metric doesn't improve for k epochs)
* Try customizing any of the 3 components (word embeddings, action choosing, combining) in clever ways.  You can create new classes that expose the same public interface and use them here (just leave your required ones untouched). Building word embeddings from characters using an RNN or convolutional layer may help.
* Try new features.  Write new classes that expose the same public interface as SimpleFeatureExtractor.  Try looking further into stack history, or more input buffer lookahead, or features based on the action sequence. The possibilities are endless.
* Within our currect interface, you can use any neural nets and static pretrained word embeddings. However, if you use pretrained contextualized embeddings from large data (e.g., BERT), you will not receive extra credits if your model ranks high. You can still try creative modifications and get extra credits for "creative solutions".

**Tests**: 
- `tests/test_parser.py:test_dev_preds_bakeoff_d5_1_english`
- `tests/test_parser.py:test_dev_preds_bakeoff_d5_2_norwegian`

**Rubric**:
English dev:
- $\geq$ 0.76: 2 points
- $\geq$ 0.77: 4 points
- $\geq$ 0.78: 6 points

English test:
- $\geq$ 0.72: 2 points
- $\geq$ 0.73: 4 points
- $\geq$ 0.74: 6 points

Norwegian dev:
- $\geq$ 0.71: 2 points
- $\geq$ 0.72: 4 points
- $\geq$ 0.73: 6 points

Norwegian test:
- $\geq$ 0.70: 2 points
- $\geq$ 0.71: 4 points
- $\geq$ 0.72: 6 points

**Extra credit**:

For both languages:
- Top five test performance in the class (4 points bonus)
- We'll also give 4 bonus points to particularly unique / creative / well-motivated solutions (with motivation to be included in the write-up).

### Using Cuda
You can use CUDA to train your network, and you should expect decent speedup if you have a GPU and the CUDA toolkit installed.
If you want to use CUDA in this assignment, change the HAVE_CUDA variable to True in constants.py, and call `.to_cuda()` on your parser. You may also need to reconfigure your Embedding layers if you didn't consider cuda before.

We are not officially supporting CUDA though. If you have problems installing or running CUDA, perhaps just use the CPU. You can post on Piazza but we cannot guarantee to help you debug it.

In [ ]:
# Set your hyperparameters here
# e.g learning rate, regularization, lr annealing, dimensionality of embeddings, number of epochs, early stopping etc.

In [ ]:
# Make your parser here
# name your TransitionParser bakeoff_parser to output your predictions below
# bakeoff_parser_en = TransitionParser(...)

# Also, choose an optimizer.
# bakeoff_optimizer_en = optim....

In [ ]:
# train for bakeoff
train_parser(bakeoff_parser_en, bakeoff_optimizer_en, en_dataset, n_epochs=5, n_train_insts=10000)

In [ ]:
# write dev output
dev_sentences = [ sentence for sentence, _ in en_dataset.dev_data ]
evaluation.output_preds("bakeoff-dev-en.preds", bakeoff_parser_en, dev_sentences)

In [ ]:
# dev performance
evaluation.compute_output_attachment("bakeoff-dev-en.preds", consts.EN_DEV_GOLD)

In [ ]:
# write test output
evaluation.output_preds("bakeoff-test-en.preds", bakeoff_parser_en, en_dataset.test_data)

In [ ]:
# Now make your norwegian parser if necessary
# name your TransitionParser bakeoff_parser to output your predictions below
# bakeoff_parser_nr = TransitionParser(...)

# Also, choose an optimizer.
# bakeoff_optimizer_nr = optim....

In [ ]:
# train for bakeoff
train_parser(bakeoff_parser_nr, bakeoff_optimizer_nr, nr_dataset, n_epochs=5, n_train_insts=10000)

In [ ]:
# write dev output
dev_sentences_nr = [ sentence for sentence, _ in nr_dataset.dev_data ]
evaluation.output_preds("bakeoff-dev-nr.preds", bakeoff_parser_nr, dev_sentences_nr)

In [ ]:
# dev performance
evaluation.compute_output_attachment("bakeoff-dev-nr.preds", consts.NR_DEV_GOLD)

In [ ]:
# write test output
evaluation.output_preds("bakeoff-test-nr.preds", bakeoff_parser_nr, nr_dataset.test_data)

# 6. Writeup (22 points)

You can start your write-up in any format you prefer (e.g., LaTeX, Markdown), but please remember to export to `pset3-writeup.pdf` upon submission. Also, you will be asked to post your Deliverable 6.2 and 6.3 on Piazza after the due date (plus late days).

**Deliverable 6.1** (6 points):

Consider a sentence of 10 words:
- In Project 1, we learned to build a text classifier. If the classifier is binary, there are $2$ possible outputs for this sentence.
- In Project 2, we learned to build a sequence tagger. If each word has four possible labels, there are $4^{10} = 1048576$ possible outputs for this sentence.
- In this project, we learned to build a parser for **projective** dependency trees. **How many possible projective dependency trees are there for our sentence of 10 words? Why? Is this output space larger than the one above?**

**Deliverable 6.2** (6 points):

Briefly describe your bakeoff design.

**Deliverable 6.3** (10 points):

You will select a research paper at ACL, EMNLP or NAACL that **uses dependency trees for some downstream task**. Summarize the paper, answering the following questions:

1. What is the task that is being solved?
2. Briefly (one sentence) explain the metric for success on this task.
3. Why are dependency features expected to help with this task?
4. How are dependency features incorporated into the solution?
5. Does the paper evaluate whether dependency features improve performance on the downstream task? If so, what is their impact? If not, why not?

You must choose a paper in the main conference (not workshops). The paper must be at least four pages long. All papers from these conferences are available for free online: https://www.aclweb.org/anthology/.